In [1]:
import time
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from ipywidgets import widgets, interact

/Users/kate/mitsuba3-scripts


In [2]:
losses = np.load('opt_tests/losses_rough.npy')
param_errors = np.load('opt_tests/param_errors_rough.npy')
param_values = np.load('opt_tests/param_opt_rough.npy')
sensor_values = np.load('opt_tests/sensor_opt_rough.npy')
sensor_ref = np.load('opt_tests/sensor_ref_rough.npy')

iteration_count = np.shape(losses)[-1]

In [3]:
# Initial values
roughness_inits = np.array([0.01, 0.1, 0.25, 0.5])

# Learning rates
learning_rates = np.array([0.005, 0.05, 0.1, 0.5])

param_ref = 0.2

In [8]:
init_select = widgets.ToggleButtons(options = roughness_inits, description = 'init: ', disabled = False)
lr_select = widgets.ToggleButtons(options = learning_rates, description = 'lr: ', disabled = False)

ui = widgets.VBox(children = [init_select, lr_select])

fig = make_subplots(rows = 3, cols = 1, vertical_spacing = 0.1,
                    subplot_titles=("Objective function", "Parameter error", "Parameter value"))

x_iter = np.array(range(iteration_count))

def scatter_plot(data, name, line_style, line_color, showlegend):
    plot = go.Scatter(
                x = x_iter,
                y = data,
                mode = 'lines',
                name = name,
                legendgroup = name,
                line = dict(
                            color = line_color,
                            dash = line_style
                       ),
                showlegend = showlegend
            )
    
    return plot

loss_plot = scatter_plot(losses[0,0,:], 'Loss', 'solid', 'blue', False)
param_error_plot = scatter_plot(param_errors[0,0,:], 'Parameter error', 'solid', 'red', False)
param_plot = scatter_plot(param_values[0,0,:], 'Opt parameter', 'solid', 'red', True)

fig.add_trace(loss_plot, row = 1, col = 1)
fig.add_trace(param_error_plot, row = 2, col = 1)
fig.add_trace(param_plot, row = 3, col = 1)

# Parameter reference value
params_ref_y = np.full(np.shape(x_iter), param_ref)
param_ref_plot = scatter_plot(params_ref_y, 'Reference parameter', 'solid', 'black', True)
fig.add_trace(param_ref_plot, row = 3, col = 1)

fig.update_layout(
    height = 600,
    title = 'Optimization stats',
    yaxis_title = 'MSE (loss)',
    yaxis_title_standoff = 0,
    yaxis2_title = 'MSE (param)',
    yaxis2_title_standoff = 0,
    xaxis_title_standoff = 0,
    xaxis2_title_standoff = 0,
    xaxis3_title = 'Iteration'
)

fig_widget = go.FigureWidget(fig)

def update(init_select, lr_select):
    i = np.where(init_select == roughness_inits)[0][0]
    lr = np.where(lr_select == learning_rates)[0][0]
    
    with fig_widget.batch_update():
        fig_widget.data[0].y = losses[i,lr,:]
        fig_widget.data[1].y = param_errors[i,lr,:]
        fig_widget.data[2].y = param_values[i,lr,:]

out = widgets.interactive_output(update, {'init_select': init_select,
                                          'lr_select': lr_select})

display(ui)
fig_widget

FigureWidget({
    'data': [{'legendgroup': 'Loss',
              'line': {'color': 'blue', 'dash': 'solid'},
…

In [7]:
import plotly.express as px

init_select = widgets.ToggleButtons(options = roughness_inits, description = 'init: ', disabled = False)
lr_select = widgets.ToggleButtons(options = learning_rates, description = 'lr: ', disabled = False)
it_slider = widgets.IntSlider(value = 0, min = 0, max = iteration_count-1, description = 'iteration: ', 
                              continuous_update = True, disabled = False,
                              layout = widgets.Layout(width = '98%'))

ui = widgets.VBox(children = [init_select, lr_select, it_slider])

theta_sensors = np.linspace(0.0, 60.0, 13)
phi_sensors   = np.array([0.0, 60.0, 180.0, 240.0])

n_theta = len(theta_sensors)
n_phi = len(phi_sensors)

x = np.shape(sensor_values)[0]
y = np.shape(sensor_values)[1]

sensor_values_im = np.reshape(sensor_values, (x, y, iteration_count, n_theta, n_phi, 4))
sensor_ref_im = np.reshape(sensor_ref, (n_theta, n_phi, 4))
sensor_err_im = sensor_values_im - sensor_ref_im

fig1 = go.Heatmap(z = np.transpose(sensor_values_im[0, 0, 0, :, :, 0]),
                  colorscale = 'gray',
                  colorbar = dict(len = 0.3, y = 0.87),
                  x = [f"{t:.0f}" for t in theta_sensors],
                  y = [f"{p:.0f}" for p in phi_sensors])

fig2 = go.Heatmap(z = np.transpose(sensor_ref_im[:, :, 0]),
                  colorscale = 'gray',
                  colorbar = dict(len = 0.3, y = 0.5),
                  x = [f"{t:.0f}" for t in theta_sensors],
                  y = [f"{p:.0f}" for p in phi_sensors])

fig3 = go.Heatmap(z = np.transpose(sensor_err_im[0, 0, 0, :, :, 0]),
                  colorscale='RdBu',
                  zmid = 0,
                  colorbar = dict(len = 0.3, y = 0.14),
                  x = [f"{t:.0f}" for t in theta_sensors],
                  y = [f"{p:.0f}" for p in phi_sensors])


fig = make_subplots(rows = 3, cols = 1,
                    vertical_spacing = 0.15,
                    subplot_titles = ('Sensor values (optimization)',
                                      'Sensor values (reference)',
                                      'Error (absolute)'))
fig.add_trace(fig1, row = 1, col = 1)
fig.add_trace(fig2, row = 2, col = 1)
fig.add_trace(fig3, row = 3, col = 1)

fig.update_layout(
    height = 750
)

for row in range(1,4):
    fig.update_xaxes(title = 'sensor theta', row = row, col = 1, title_standoff = 0)
    fig.update_yaxes(title = 'sensor phi', row = row, col = 1, title_standoff = 0)

fig_widget = go.FigureWidget(fig)

def update(init_select, lr_select, it_slider):
    i = np.where(init_select == roughness_inits)[0][0]
    lr = np.where(lr_select == learning_rates)[0][0]
    
    with fig_widget.batch_update():
        fig_widget.data[0].z = np.transpose(sensor_values_im[i, lr, it_slider, :, :, 0])
        fig_widget.data[2].z = np.transpose(sensor_err_im[i, lr, it_slider, :, :, 0])

out = widgets.interactive_output(update, {'init_select': init_select,
                                          'lr_select': lr_select,
                                          'it_slider': it_slider})

display(ui)
fig_widget

FigureWidget({
    'data': [{'colorbar': {'len': 0.3, 'y': 0.87},
              'colorscale': [[0.0, 'rgb(0, 0…